# yo-yo-maskr

In [4]:
from yoyomaskr import llm_find_entities

with open('sample_text.txt', 'r') as file:
    sample_text = file.read()
result = llm_find_entities(sample_text)
for k, v in result.items():
    print(f'{k}: {v}')

#person_1#: ['Marie Müller', 'Marie']
#person_2#: ['Thomas Müller', 'Thomas']
#person_3#: ['Anna Schmidt', 'Anna']
#person_4#: ['Lukas Hoffmann', 'Lukas']
#person_5#: ['Peter Lehmann', 'Peter']
#person_6#: ['Karl Weber', 'Karl']
#person_7#: ['Karla Fischer', 'Karla']
#person_8#: ['Paul König', 'Paul']
#person_9#: ['Andreas Schneider', 'Andreas']
#person_10#: ['Julia Wagner', 'Julia']
#person_11#: ['Frank']
#person_12#: ['Sabine Schmidt', 'Sabine']
#person_13#: ['Klaus Neumann', 'Klaus']
#place_1#: ['Hamburg']
#place_2#: ['Berlin']
#place_3#: ['München']
#place_4#: ['Italien']
#place_5#: ['Rom']
#place_6#: ['Mailand']
#place_7#: ['Venedig']
#place_8#: ['Friedrichstraße 15']
#place_9#: ['Charlottenburg']
#place_10#: ['Potsdam']
#place_11#: ['Sanssouci']
#place_12#: ['Kurfürstendamm']
#place_13#: ['Dresden']


#person_1#: ['Marie Müller', 'Marie']
#person_2#: ['Thomas', 'Thomas Müller']
#person_3#: ['Anna Schmidt', 'Anna']
#person_4#: ['Lukas Hoffmann', 'Lukas']
#person_5#: ['Peter Lehmann', 'Peter']
#person_6#: ['Karl Weber', 'Karl']
#person_7#: ['Karla Fischer', 'Karla']
#person_8#: ['Paul König', 'Paul']
#person_9#: ['Andreas Schneider', 'Andreas']
#person_10#: ['Julia Wagner', 'Julia']
#person_11#: ['Frank']
#person_12#: ['Sabine Schmidt', 'Sabine']
#person_13#: ['Klaus Neumann', 'Klaus']
#place_1#: ['Hamburg']
#place_2#: ['Berlin']
#place_3#: ['München']
#place_4#: ['Italien']
#place_5#: ['Rom']
#place_6#: ['Mailand']
#place_7#: ['Venedig']
#place_8#: ['Friedrichstraße 15']
#place_9#: ['Charlottenburg']
#place_10#: ['Potsdam']
#place_11#: ['Sanssouci']
#place_12#: ['Kurfürstendamm']
#place_13#: ['Dresden']
original_text: Es war ein kühler Herbstmorgen, als Marie Müller mit ihrem Bruder Thomas in Hamburg auf den Zug nach Berlin wartete. Dort wollten sie ihre Freunde Anna Schmidt und Luka

{'#person_1#': ['Marie Müller', 'Marie'],
 '#person_2#': ['Thomas Müller', 'Thomas'],
 '#person_3#': ['Anna Schmidt', 'Anna'],
 '#person_4#': ['Lukas Hoffmann', 'Lukas'],
 '#person_5#': ['Peter Lehmann', 'Peter'],
 '#person_6#': ['Karl Weber', 'Karl'],
 '#person_7#': ['Karla Fischer', 'Karla'],
 '#person_8#': ['Paul König', 'Paul'],
 '#person_9#': ['Andreas Schneider', 'Andreas'],
 '#person_10#': ['Julia Wagner', 'Julia'],
 '#person_11#': ['Frank'],
 '#person_12#': ['Sabine Schmidt', 'Sabine'],
 '#person_13#': ['Klaus Neumann', 'Klaus'],
 '#place_1#': ['Hamburg'],
 '#place_2#': ['Berlin'],
 '#place_3#': ['München'],
 '#place_4#': ['Italien'],
 '#place_5#': ['Rom'],
 '#place_6#': ['Mailand'],
 '#place_7#': ['Venedig'],
 '#place_8#': ['Friedrichstraße 15'],
 '#place_9#': ['Charlottenburg'],
 '#place_10#': ['Potsdam'],
 '#place_11#': ['Sanssouci'],
 '#place_12#': ['Kurfürstendamm'],
 '#place_13#': ['Dresden'],
 'original_text': ['E',
  's',
  ' ',
  'w',
  'a',
  'r',
  ' ',
  'e',
  'i',